## Updated Lens Generation with Edge Additions
- Here, we make lenses that have consistent sheared camera view, we also add edges.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import plotly
from lensless_helpers import *
from cleanplots import *

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter

### Make PSF flat images

In [ ]:
diffuser_psf = skimage.io.imread('/home/your_username/EncodingInformation/imager_experiments/psfs/diffuser_psf.png')
diffuser_psf = diffuser_psf[:,:,1]
diffuser_resize = diffuser_psf[200:500, 250:550]
diffuser_resize = resize(diffuser_resize, (400, 400), anti_aliasing=True)  #resize(diffuser_psf, (28, 28))
diffuser_region = diffuser_resize[:32*4, :32*4]
diffuser_region /=  np.sum(diffuser_region)

In [ ]:
import matplotlib.colors as mcolors 

lime = mcolors.LinearSegmentedColormap.from_list("custom_cmap", ['black', 'lime'])

In [ ]:
# green psfs 
diffuser_psf = load_diffuser_32()
four_psf = load_four_lens_32()
one_psf = load_single_lens_32()
plt.figure(figsize=(10, 4))
plt.subplot(1,3,1)
plt.imshow(one_psf, cmap=lime, interpolation='spline36')
plt.title('Single Lens')
# clear the axes
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.subplot(1,3,2)
plt.imshow(four_psf, cmap=lime, interpolation='spline36')
plt.title('Four Lenses')
# clear the axes
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.subplot(1,3,3)
plt.imshow(diffuser_region, cmap=lime, interpolation='spline36')
plt.title('Diffuser')
# clear the axes
plt.gca().set_xticks([])
plt.gca().set_yticks([])

#plt.savefig('01_11_2024_figures/green_planar_psfs.pdf', transparent=True, bbox_inches='tight')

In [ ]:
# load the PSFs

diffuser_psf = load_diffuser_32()
four_psf = load_four_lens_32()
one_psf = load_single_lens_32()
plt.figure(figsize=(10, 4))
plt.subplot(1,3,1)
plt.imshow(one_psf, cmap='inferno', interpolation='spline36')
plt.title('Single Lens')
# clear the axes
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.subplot(1,3,2)
plt.imshow(four_psf, cmap='inferno', interpolation='spline36')
plt.title('Four Lenses')
# clear the axes
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.subplot(1,3,3)
plt.imshow(diffuser_region, cmap='inferno', interpolation='spline36')
plt.title('Diffuser')
# clear the axes
plt.gca().set_xticks([])
plt.gca().set_yticks([])

#plt.savefig('01_11_2024_figures/planar_psfs.pdf', transparent=True, bbox_inches='tight')

In [ ]:
z_four_lens = np.zeros((500, 500))
z_four_lens[375, 125] = 1 
z_four_lens[220, 175] = 1
z_four_lens[100, 250] = 1
z_four_lens[350, 380] = 1
z_four_lens = z_four_lens * 1
z_four_lens = gaussian_filter(z_four_lens, sigma=40)
z_four_lens = np.rot90(z_four_lens, k=-1)
plt.imshow(z_four_lens)
plt.colorbar()

## Four Lens

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = z_four_lens * -1
# make a ramp object for testing
# z_diffuser = np.zeros((500, 500))
# for i in range(500):
#     z_diffuser[i, :] = i / 500
    
# make circular instead of square
z_diffuser_circle = np.copy(z_diffuser) # make a copy so that the edge code below can find the correct height
z_diffuser_circle[r > 1] = np.nan
opacity_mask = (r < 1).astype(np.float32)
lighting = dict(ambient=0.1, fresnel=0, roughness=0.2, specular=.065) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
opacity=1
# color_name= 'blue'
color1 = 'rgb(80, 110, 255)'
color2 = 'rgb(150, 180, 255)'
# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x,
    y=y,
    z=z_diffuser_circle,
    colorscale=[[0, color1], [1, color2]], 
    showscale=False, 
    opacity=opacity,     
    lighting=lighting,
      lightposition=lightposition,
)
# Define the cylinder geometry
theta = np.linspace(0, 2*np.pi, 500)
z_cylinder = np.linspace(-.1, .1, 250)  # Adjust -1 and 1 to set the height of the cylinder
theta_grid, z_grid = np.meshgrid(theta, z_cylinder)
x_cylinder = np.cos(theta_grid)
y_cylinder = np.sin(theta_grid)
# change the height to match the height of the phase mask
for j in range(z_grid.shape[1]):
    # find the height of the phase mask at this xy location
    y_pos, x_pos = y_cylinder[0, j], x_cylinder[0, j]  # swap x_cylinder and y_cylinder to match x and y axes correctly
    # Convert from [-1, 1] to [0, size) and ensure index is within bounds
    x_index = np.clip(int((x_pos + 1) / 2 * (x.size - 1)), 0, x.size - 1)
    y_index = np.clip(int((y_pos + 1) / 2 * (y.size - 1)), 0, y.size - 1)
    # find the corresponding index in the phase mask
    phase_mask_height = z_diffuser[y_index, x_index]
    z_grid[:, j] = np.linspace(phase_mask_height, .1, 250)  # Adjust the end height (.1) as needed
# Assuming the cylinder's radius is 1
color2 = 'rgb(120, 150, 255)'
color1 = 'rgb(180, 210, 255)'
fig.add_surface(
    x=x_cylinder, 
    y=y_cylinder, 
    z=z_grid, 
    colorscale=[[0, color1], [1, color2]],  # Change color as needed
    showscale=False, 
    opacity=opacity,
    # lighting=lighting,
    lightposition=lightposition
)
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor=None,
    plot_bgcolor=None
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
#fig.write_image('01_11_2024_figures/02_14_four_lens_render_test.png', scale=1)

In [ ]:

# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = z_four_lens * -1
# make circular instead of square
z_diffuser[r > 1] = np.nan 

lighting = dict(ambient=0, fresnel=3, roughness=0.75, specular=0.5) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
color_name = 'lightblue' 
opacity = 0.95

# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x, 
    y=y, 
    z=z_diffuser, 
    colorscale=[[0, color_name], [1, color_name]],
    showscale=False, 
    opacity=opacity, lighting=lighting, lightposition=lightposition
)

# Update layout for camera, lighting, and axes
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor='white',
    plot_bgcolor='white'
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
#fig.write_image('01_11_2024_figures/02_12_four_lens_render.pdf', scale=10)

## Single Lens

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = gaussian_filter(r, sigma=15)
# make a ramp object for testing
# z_diffuser = np.zeros((500, 500))
# for i in range(500):
#     z_diffuser[i, :] = i / 500
    
# make circular instead of square
z_diffuser_circle = np.copy(z_diffuser) # make a copy so that the edge code below can find the correct height
z_diffuser_circle[r > 1] = np.nan
opacity_mask = (r < 1).astype(np.float32)
lighting = dict(ambient=0.1, fresnel=0, roughness=0.2, specular=.065) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
opacity=1
# color_name= 'blue'
color1 = 'rgb(80, 110, 255)'
color2 = 'rgb(150, 180, 255)'
# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x,
    y=y,
    z=z_diffuser_circle,
    colorscale=[[0, color1], [1, color2]], 
    showscale=False, 
    opacity=opacity,     
    lighting=lighting,
      lightposition=lightposition,
)
# Define the cylinder geometry
theta = np.linspace(0, 2*np.pi, 500)
z_cylinder = np.linspace(-.1, .1, 250)  # Adjust -1 and 1 to set the height of the cylinder
theta_grid, z_grid = np.meshgrid(theta, z_cylinder)
x_cylinder = np.cos(theta_grid)
y_cylinder = np.sin(theta_grid)
# change the height to match the height of the phase mask
for j in range(z_grid.shape[1]):
    # find the height of the phase mask at this xy location
    y_pos, x_pos = y_cylinder[0, j], x_cylinder[0, j]  # swap x_cylinder and y_cylinder to match x and y axes correctly
    # Convert from [-1, 1] to [0, size) and ensure index is within bounds
    x_index = np.clip(int((x_pos + 1) / 2 * (x.size - 1)), 0, x.size - 1)
    y_index = np.clip(int((y_pos + 1) / 2 * (y.size - 1)), 0, y.size - 1)
    # find the corresponding index in the phase mask
    phase_mask_height = z_diffuser[y_index, x_index]
    z_grid[:, j] = np.linspace(phase_mask_height, .1, 250)  # Adjust the end height (.1) as needed
# Assuming the cylinder's radius is 1
color2 = 'rgb(120, 150, 255)'
color1 = 'rgb(180, 210, 255)'
fig.add_surface(
    x=x_cylinder, 
    y=y_cylinder, 
    z=z_grid, 
    colorscale=[[0, color1], [1, color2]],  # Change color as needed
    showscale=False, 
    opacity=opacity,
    # lighting=lighting,
    lightposition=lightposition
)
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor=None,
    plot_bgcolor=None
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
#fig.write_image('01_11_2024_figures/02_14_one_lens_render_test.png', scale=1)

## Make lens renders (IDEAL) 

In [ ]:
folder = 'e2e_full_volume_phase_masks/high_res_dino/'
initial = np.load(folder + 'checkpoint_0.npy')
middle = np.load(folder +'checkpoint_120.npy')
final = np.load(folder + 'checkpoint_1080.npy')


In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = -15 * np.copy(initial) 
# make a ramp object for testing
# z_diffuser = np.zeros((500, 500))
# for i in range(500):
#     z_diffuser[i, :] = i / 500
    
# make circular instead of square
z_diffuser_circle = np.copy(z_diffuser) # make a copy so that the edge code below can find the correct height
z_diffuser_circle[r > 1] = np.nan
opacity_mask = (r < 1).astype(np.float32)
lighting = dict(ambient=0.1, fresnel=0, roughness=0.2, specular=.065) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
opacity=1
# color_name= 'blue'
color1 = 'rgb(80, 110, 255)'
color2 = 'rgb(150, 180, 255)'
# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x,
    y=y,
    z=z_diffuser_circle,
    colorscale=[[0, color1], [1, color2]], 
    showscale=False, 
    opacity=opacity,     
    lighting=lighting,
      lightposition=lightposition,
)
# Define the cylinder geometry
theta = np.linspace(0, 2*np.pi, 500)
z_cylinder = np.linspace(-.1, .1, 250)  # Adjust -1 and 1 to set the height of the cylinder
theta_grid, z_grid = np.meshgrid(theta, z_cylinder)
x_cylinder = np.cos(theta_grid)
y_cylinder = np.sin(theta_grid)
# change the height to match the height of the phase mask
for j in range(z_grid.shape[1]):
    # find the height of the phase mask at this xy location
    y_pos, x_pos = y_cylinder[0, j], x_cylinder[0, j]  # swap x_cylinder and y_cylinder to match x and y axes correctly
    # Convert from [-1, 1] to [0, size) and ensure index is within bounds
    x_index = np.clip(int((x_pos + 1) / 2 * (x.size - 1)), 0, x.size - 1)
    y_index = np.clip(int((y_pos + 1) / 2 * (y.size - 1)), 0, y.size - 1)
    # find the corresponding index in the phase mask
    phase_mask_height = z_diffuser[y_index, x_index]
    z_grid[:, j] = np.linspace(phase_mask_height, .1, 250)  # Adjust the end height (.1) as needed
# Assuming the cylinder's radius is 1
color2 = 'rgb(120, 150, 255)'
color1 = 'rgb(180, 210, 255)'
fig.add_surface(
    x=x_cylinder, 
    y=y_cylinder, 
    z=z_grid, 
    colorscale=[[0, color1], [1, color2]],  # Change color as needed
    showscale=False, 
    opacity=opacity,
    # lighting=lighting,
    lightposition=lightposition
)
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor=None,
    plot_bgcolor=None
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
#fig.write_image('01_11_2024_figures/02_14_dino_initial_lens_render_test.png', scale=1)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = -15 * np.copy(middle) 
# make a ramp object for testing
# z_diffuser = np.zeros((500, 500))
# for i in range(500):
#     z_diffuser[i, :] = i / 500
    
# make circular instead of square
z_diffuser_circle = np.copy(z_diffuser) # make a copy so that the edge code below can find the correct height
z_diffuser_circle[r > 1] = np.nan
opacity_mask = (r < 1).astype(np.float32)
lighting = dict(ambient=0.1, fresnel=0, roughness=0.2, specular=.065) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
opacity=1
# color_name= 'blue'
color1 = 'rgb(80, 110, 255)'
color2 = 'rgb(150, 180, 255)'
# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x,
    y=y,
    z=z_diffuser_circle,
    colorscale=[[0, color1], [1, color2]], 
    showscale=False, 
    opacity=opacity,     
    lighting=lighting,
      lightposition=lightposition,
)
# Define the cylinder geometry
theta = np.linspace(0, 2*np.pi, 500)
z_cylinder = np.linspace(-.1, .1, 250)  # Adjust -1 and 1 to set the height of the cylinder
theta_grid, z_grid = np.meshgrid(theta, z_cylinder)
x_cylinder = np.cos(theta_grid)
y_cylinder = np.sin(theta_grid)
# change the height to match the height of the phase mask
for j in range(z_grid.shape[1]):
    # find the height of the phase mask at this xy location
    y_pos, x_pos = y_cylinder[0, j], x_cylinder[0, j]  # swap x_cylinder and y_cylinder to match x and y axes correctly
    # Convert from [-1, 1] to [0, size) and ensure index is within bounds
    x_index = np.clip(int((x_pos + 1) / 2 * (x.size - 1)), 0, x.size - 1)
    y_index = np.clip(int((y_pos + 1) / 2 * (y.size - 1)), 0, y.size - 1)
    # find the corresponding index in the phase mask
    phase_mask_height = z_diffuser[y_index, x_index]
    z_grid[:, j] = np.linspace(phase_mask_height, .1, 250)  # Adjust the end height (.1) as needed
# Assuming the cylinder's radius is 1
color2 = 'rgb(120, 150, 255)'
color1 = 'rgb(180, 210, 255)'
fig.add_surface(
    x=x_cylinder, 
    y=y_cylinder, 
    z=z_grid, 
    colorscale=[[0, color1], [1, color2]],  # Change color as needed
    showscale=False, 
    opacity=opacity,
    # lighting=lighting,
    lightposition=lightposition
)
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor=None,
    plot_bgcolor=None
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
#fig.write_image('01_11_2024_figures/02_14_dino_middle_lens_render_test.png', scale=1)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = -15 * np.copy(final) 
# make a ramp object for testing
# z_diffuser = np.zeros((500, 500))
# for i in range(500):
#     z_diffuser[i, :] = i / 500
    
# make circular instead of square
z_diffuser_circle = np.copy(z_diffuser) # make a copy so that the edge code below can find the correct height
z_diffuser_circle[r > 1] = np.nan
opacity_mask = (r < 1).astype(np.float32)
lighting = dict(ambient=0.1, fresnel=0, roughness=0.2, specular=.065) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
opacity=1
# color_name= 'blue'
color1 = 'rgb(80, 110, 255)'
color2 = 'rgb(150, 180, 255)'
# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x,
    y=y,
    z=z_diffuser_circle,
    colorscale=[[0, color1], [1, color2]], 
    showscale=False, 
    opacity=opacity,     
    lighting=lighting,
      lightposition=lightposition,
)
# Define the cylinder geometry
theta = np.linspace(0, 2*np.pi, 500)
z_cylinder = np.linspace(-.1, .1, 250)  # Adjust -1 and 1 to set the height of the cylinder
theta_grid, z_grid = np.meshgrid(theta, z_cylinder)
x_cylinder = np.cos(theta_grid)
y_cylinder = np.sin(theta_grid)
# change the height to match the height of the phase mask
for j in range(z_grid.shape[1]):
    # find the height of the phase mask at this xy location
    y_pos, x_pos = y_cylinder[0, j], x_cylinder[0, j]  # swap x_cylinder and y_cylinder to match x and y axes correctly
    # Convert from [-1, 1] to [0, size) and ensure index is within bounds
    x_index = np.clip(int((x_pos + 1) / 2 * (x.size - 1)), 0, x.size - 1)
    y_index = np.clip(int((y_pos + 1) / 2 * (y.size - 1)), 0, y.size - 1)
    # find the corresponding index in the phase mask
    phase_mask_height = z_diffuser[y_index, x_index]
    z_grid[:, j] = np.linspace(phase_mask_height, .1, 250)  # Adjust the end height (.1) as needed
# Assuming the cylinder's radius is 1
color2 = 'rgb(120, 150, 255)'
color1 = 'rgb(180, 210, 255)'
fig.add_surface(
    x=x_cylinder, 
    y=y_cylinder, 
    z=z_grid, 
    colorscale=[[0, color1], [1, color2]],  # Change color as needed
    showscale=False, 
    opacity=opacity,
    # lighting=lighting,
    lightposition=lightposition
)
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor=None,
    plot_bgcolor=None
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
#fig.write_image('01_11_2024_figures/02_14_dino_final_lens_render_test.png', scale=1)

## Make lens renders for IDEAL

In [ ]:
folder = 'e2e_full_volume_phase_masks/phase_masks_high_res/'
initial = np.load(folder + 'checkpoint_0.npy')
middle = np.load(folder +'checkpoint_300.npy')
final = np.load(folder + 'checkpoint_5660.npy')


In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = -15 * np.copy(initial) 
# make a ramp object for testing
# z_diffuser = np.zeros((500, 500))
# for i in range(500):
#     z_diffuser[i, :] = i / 500
    
# make circular instead of square
z_diffuser_circle = np.copy(z_diffuser) # make a copy so that the edge code below can find the correct height
z_diffuser_circle[r > 1] = np.nan
opacity_mask = (r < 1).astype(np.float32)
lighting = dict(ambient=0.1, fresnel=0, roughness=0.2, specular=.065) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
opacity=1
# color_name= 'blue'
color1 = 'rgb(80, 110, 255)'
color2 = 'rgb(150, 180, 255)'
# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x,
    y=y,
    z=z_diffuser_circle,
    colorscale=[[0, color1], [1, color2]], 
    showscale=False, 
    opacity=opacity,     
    lighting=lighting,
      lightposition=lightposition,
)
# Define the cylinder geometry
theta = np.linspace(0, 2*np.pi, 500)
z_cylinder = np.linspace(-.1, .1, 250)  # Adjust -1 and 1 to set the height of the cylinder
theta_grid, z_grid = np.meshgrid(theta, z_cylinder)
x_cylinder = np.cos(theta_grid)
y_cylinder = np.sin(theta_grid)
# change the height to match the height of the phase mask
for j in range(z_grid.shape[1]):
    # find the height of the phase mask at this xy location
    y_pos, x_pos = y_cylinder[0, j], x_cylinder[0, j]  # swap x_cylinder and y_cylinder to match x and y axes correctly
    # Convert from [-1, 1] to [0, size) and ensure index is within bounds
    x_index = np.clip(int((x_pos + 1) / 2 * (x.size - 1)), 0, x.size - 1)
    y_index = np.clip(int((y_pos + 1) / 2 * (y.size - 1)), 0, y.size - 1)
    # find the corresponding index in the phase mask
    phase_mask_height = z_diffuser[y_index, x_index]
    z_grid[:, j] = np.linspace(phase_mask_height, .1, 250)  # Adjust the end height (.1) as needed
# Assuming the cylinder's radius is 1
color2 = 'rgb(120, 150, 255)'
color1 = 'rgb(180, 210, 255)'
fig.add_surface(
    x=x_cylinder, 
    y=y_cylinder, 
    z=z_grid, 
    colorscale=[[0, color1], [1, color2]],  # Change color as needed
    showscale=False, 
    opacity=opacity,
    # lighting=lighting,
    lightposition=lightposition
)
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor=None,
    plot_bgcolor=None
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
#fig.write_image('01_11_2024_figures/02_14_initial_lens_render_test.png', scale=1)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = -15 * np.copy(middle) 
# make a ramp object for testing
# z_diffuser = np.zeros((500, 500))
# for i in range(500):
#     z_diffuser[i, :] = i / 500
    
# make circular instead of square
z_diffuser_circle = np.copy(z_diffuser) # make a copy so that the edge code below can find the correct height
z_diffuser_circle[r > 1] = np.nan
opacity_mask = (r < 1).astype(np.float32)
lighting = dict(ambient=0.1, fresnel=0, roughness=0.2, specular=.065) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
opacity=1
# color_name= 'blue'
color1 = 'rgb(80, 110, 255)'
color2 = 'rgb(150, 180, 255)'
# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x,
    y=y,
    z=z_diffuser_circle,
    colorscale=[[0, color1], [1, color2]], 
    showscale=False, 
    opacity=opacity,     
    lighting=lighting,
      lightposition=lightposition,
)
# Define the cylinder geometry
theta = np.linspace(0, 2*np.pi, 500)
z_cylinder = np.linspace(-.1, .1, 250)  # Adjust -1 and 1 to set the height of the cylinder
theta_grid, z_grid = np.meshgrid(theta, z_cylinder)
x_cylinder = np.cos(theta_grid)
y_cylinder = np.sin(theta_grid)
# change the height to match the height of the phase mask
for j in range(z_grid.shape[1]):
    # find the height of the phase mask at this xy location
    y_pos, x_pos = y_cylinder[0, j], x_cylinder[0, j]  # swap x_cylinder and y_cylinder to match x and y axes correctly
    # Convert from [-1, 1] to [0, size) and ensure index is within bounds
    x_index = np.clip(int((x_pos + 1) / 2 * (x.size - 1)), 0, x.size - 1)
    y_index = np.clip(int((y_pos + 1) / 2 * (y.size - 1)), 0, y.size - 1)
    # find the corresponding index in the phase mask
    phase_mask_height = z_diffuser[y_index, x_index]
    z_grid[:, j] = np.linspace(phase_mask_height, .1, 250)  # Adjust the end height (.1) as needed
# Assuming the cylinder's radius is 1
color2 = 'rgb(120, 150, 255)'
color1 = 'rgb(180, 210, 255)'
fig.add_surface(
    x=x_cylinder, 
    y=y_cylinder, 
    z=z_grid, 
    colorscale=[[0, color1], [1, color2]],  # Change color as needed
    showscale=False, 
    opacity=opacity,
    # lighting=lighting,
    lightposition=lightposition
)
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor=None,
    plot_bgcolor=None
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
#fig.write_image('01_11_2024_figures/02_14_middle_lens_render_test.png', scale=1)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = -15 * np.copy(final) 
# make a ramp object for testing
# z_diffuser = np.zeros((500, 500))
# for i in range(500):
#     z_diffuser[i, :] = i / 500
    
# make circular instead of square
z_diffuser_circle = np.copy(z_diffuser) # make a copy so that the edge code below can find the correct height
z_diffuser_circle[r > 1] = np.nan
opacity_mask = (r < 1).astype(np.float32)
lighting = dict(ambient=0.1, fresnel=0, roughness=0.2, specular=.065) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
opacity=1
# color_name= 'blue'
color1 = 'rgb(80, 110, 255)'
color2 = 'rgb(150, 180, 255)'
# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x,
    y=y,
    z=z_diffuser_circle,
    colorscale=[[0, color1], [1, color2]], 
    showscale=False, 
    opacity=opacity,     
    lighting=lighting,
      lightposition=lightposition,
)
# Define the cylinder geometry
theta = np.linspace(0, 2*np.pi, 500)
z_cylinder = np.linspace(-.1, .1, 250)  # Adjust -1 and 1 to set the height of the cylinder
theta_grid, z_grid = np.meshgrid(theta, z_cylinder)
x_cylinder = np.cos(theta_grid)
y_cylinder = np.sin(theta_grid)
# change the height to match the height of the phase mask
for j in range(z_grid.shape[1]):
    # find the height of the phase mask at this xy location
    y_pos, x_pos = y_cylinder[0, j], x_cylinder[0, j]  # swap x_cylinder and y_cylinder to match x and y axes correctly
    # Convert from [-1, 1] to [0, size) and ensure index is within bounds
    x_index = np.clip(int((x_pos + 1) / 2 * (x.size - 1)), 0, x.size - 1)
    y_index = np.clip(int((y_pos + 1) / 2 * (y.size - 1)), 0, y.size - 1)
    # find the corresponding index in the phase mask
    phase_mask_height = z_diffuser[y_index, x_index]
    z_grid[:, j] = np.linspace(phase_mask_height, .1, 250)  # Adjust the end height (.1) as needed
# Assuming the cylinder's radius is 1
color2 = 'rgb(120, 150, 255)'
color1 = 'rgb(180, 210, 255)'
fig.add_surface(
    x=x_cylinder, 
    y=y_cylinder, 
    z=z_grid, 
    colorscale=[[0, color1], [1, color2]],  # Change color as needed
    showscale=False, 
    opacity=opacity,
    # lighting=lighting,
    lightposition=lightposition
)
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor=None,
    plot_bgcolor=None
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
#fig.write_image('01_11_2024_figures/02_14_final_lens_render_test.png', scale=1)

## Diffuser without black aperture and smaller scale features (zoomed in)
Eventually want to make it a circle, so want to have the edges on the right side facing out and the top side facing out (towards the right)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = -1 * gaussian_filter(np.random.rand(y.size, x.size), sigma=25) # make it taller by multiplying by value less than 1

    
# make circular instead of square
z_diffuser_circle = np.copy(z_diffuser) # make a copy so that the edge code below can find the correct height
z_diffuser_circle[r > 1] = np.nan
opacity_mask = (r < 1).astype(np.float32)
lighting = dict(ambient=0.1, fresnel=0, roughness=0.2, specular=.065) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
opacity=1
# color_name= 'blue'
color1 = 'rgb(80, 110, 255)'
color2 = 'rgb(150, 180, 255)'
# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x,
    y=y,
    z=z_diffuser_circle,
    colorscale=[[0, color1], [1, color2]], 
    showscale=False, 
    opacity=opacity,     
    lighting=lighting,
      lightposition=lightposition,
)
# Define the cylinder geometry
theta = np.linspace(0, 2*np.pi, 500)
z_cylinder = np.linspace(-.1, .1, 250)  # Adjust -1 and 1 to set the height of the cylinder
theta_grid, z_grid = np.meshgrid(theta, z_cylinder)
x_cylinder = np.cos(theta_grid)
y_cylinder = np.sin(theta_grid)
# change the height to match the height of the phase mask
for j in range(z_grid.shape[1]):
    # find the height of the phase mask at this xy location
    y_pos, x_pos = y_cylinder[0, j], x_cylinder[0, j]  # swap x_cylinder and y_cylinder to match x and y axes correctly
    # Convert from [-1, 1] to [0, size) and ensure index is within bounds
    x_index = np.clip(int((x_pos + 1) / 2 * (x.size - 1)), 0, x.size - 1)
    y_index = np.clip(int((y_pos + 1) / 2 * (y.size - 1)), 0, y.size - 1)
    # find the corresponding index in the phase mask
    phase_mask_height = z_diffuser[y_index, x_index]
    z_grid[:, j] = np.linspace(phase_mask_height, .1, 250)  # Adjust the end height (.1) as needed
# Assuming the cylinder's radius is 1
color2 = 'rgb(120, 150, 255)'
color1 = 'rgb(180, 210, 255)'
fig.add_surface(
    x=x_cylinder, 
    y=y_cylinder, 
    z=z_grid, 
    colorscale=[[0, color1], [1, color2]],  # Change color as needed
    showscale=False, 
    opacity=opacity,
    # lighting=lighting,
    lightposition=lightposition
)
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor=None,
    plot_bgcolor=None
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
#fig.write_image('01_11_2024_figures/02_14_diffuser_render_test.png', scale=1)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = gaussian_filter(np.random.rand(y.size, x.size), sigma=25) # make it taller by multiplying by value less than 1
# make circular instead of square
z_diffuser[r > 1] = np.nan 

lighting = dict(ambient=0, fresnel=3, roughness=0.75, specular=0.5) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
color_name = 'lightblue' #'white' #'lightblue'
opacity=0.95

# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x, 
    y=y, 
    z=z_diffuser, 
    colorscale=[[0, color_name], [1, color_name]], 
    showscale=False, 
    opacity=opacity, lighting=lighting, lightposition=lightposition
)

# Update layout for camera, lighting, and axes
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor='white',
    plot_bgcolor='white'
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
#fig.write_image('01_11_2024_figures/02_12_diffuser_render.pdf', scale=10)